# Dataset Splitting and Merging

# Dataset Splitting and Merging

Split validated synthetic PII data into 80-20 train/test sets and combine with the original ai4privacy English-only
dataset splits.

Following HuggingFace Dataset Upload Guide: https://huggingface.co/docs/hub/en/datasets-upload-guide-llm

Output structure: - Combined train: ai4privacy train + 80% synthetic - Combined test: ai4privacy validation + 20%
synthetic


## Imports and Configuration


In [25]:
import json
from collections import Counter
from pathlib import Path
from typing import Any

from datasets import (
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    Value,
    load_from_disk,
)
from sklearn.model_selection import train_test_split

# Configuration
AI4PRIVACY_PATH: Path = Path("./data/ai4privacy_english_only")
SYNTHETIC_VALIDATED_PATH: Path = Path("./data/validation/validated_samples.jsonl")
OUTPUT_PATH: Path = Path("./data/combined_dataset")

# HuggingFace Hub configuration (set to None if not pushing to Hub)
HF_REPO_ID: str | None = "Ari-S-123/better-english-pii-anonymizer"  # e.g., "your-username/pii-detection-combined"
HF_PRIVATE: bool = False  # Set to False for public datasets

# Split configuration
SYNTHETIC_TRAIN_RATIO: float = 0.80
SYNTHETIC_TEST_RATIO: float = 0.20
RANDOM_SEED: int = 69

# Memory-efficient upload settings (per HF guide recommendations)
MAX_SHARD_SIZE: str = "500MB"  # Recommended for large datasets

print("Configuration:")
print(f"  ai4privacy dataset: {AI4PRIVACY_PATH}")
print(f"  Synthetic validated: {SYNTHETIC_VALIDATED_PATH}")
print(f"  Output directory: {OUTPUT_PATH}")
print(f"  HuggingFace repo: {HF_REPO_ID or 'Not configured (local only)'}")
print(f"  Synthetic split ratio: {SYNTHETIC_TRAIN_RATIO:.0%} train / {SYNTHETIC_TEST_RATIO:.0%} test")
print(f"  Random seed: {RANDOM_SEED}")
print(f"  Max shard size: {MAX_SHARD_SIZE}")

Configuration:
  ai4privacy dataset: data\ai4privacy_english_only
  Synthetic validated: data\validation\validated_samples.jsonl
  Output directory: data\combined_dataset
  HuggingFace repo: Ari-S-123/better-english-pii-anonymizer
  Synthetic split ratio: 80% train / 20% test
  Random seed: 69
  Max shard size: 500MB


## Define Explicit Features Schema

Define explicit Features schema for the unified dataset.


In [26]:
# Define the schema for privacy_mask entries (nested structure)
# This matches the ai4privacy format: {label, start, end, value, label_index}
PRIVACY_MASK_FEATURE = {
    "label": Value("string"),
    "start": Value("int64"),
    "end": Value("int64"),
    "value": Value("string"),
    "label_index": Value("int64"),
}

# Define the complete unified schema
UNIFIED_FEATURES = Features({
    # Core fields (required for NER training)
    "source_text": Value("string"),
    "privacy_mask": Sequence(PRIVACY_MASK_FEATURE),
    
    # Synthetic-specific metadata (nullable for ai4privacy records)
    "feature_dimension": Value("string"),
    "seed_pii_type": Value("string"),
    "seed_pii_value": Value("string"),
    "seed_pii_locale": Value("string"),
    "scenario": Value("string"),
    "type_variant": Value("string"),
    "generation_id": Value("string"),
    
    # Source identification
    "data_source": Value("string"),
    
    # Language/locale metadata
    "language": Value("string"),
    "region": Value("string"),
    "script": Value("string"),
})

print("Unified Features schema defined:")
for feature_name, feature_type in UNIFIED_FEATURES.items():
    print(f"  {feature_name}: {feature_type}")

Unified Features schema defined:
  source_text: Value('string')
  privacy_mask: {'label': List(Value('string')), 'start': List(Value('int64')), 'end': List(Value('int64')), 'value': List(Value('string')), 'label_index': List(Value('int64'))}
  feature_dimension: Value('string')
  seed_pii_type: Value('string')
  seed_pii_value: Value('string')
  seed_pii_locale: Value('string')
  scenario: Value('string')
  type_variant: Value('string')
  generation_id: Value('string')
  data_source: Value('string')
  language: Value('string')
  region: Value('string')
  script: Value('string')


## Load ai4privacy English-Only Dataset

Load the pre-filtered English-only ai4privacy dataset from disk.


In [27]:
ai4privacy_dataset: DatasetDict = load_from_disk(str(AI4PRIVACY_PATH))

print("ai4privacy English-only dataset loaded:")
print(f"  Train split: {len(ai4privacy_dataset['train']):,} examples")
print(f"  Validation split: {len(ai4privacy_dataset['validation']):,} examples")
print(f"\nOriginal features: {list(ai4privacy_dataset['train'].features.keys())}")

# Display sample record structure
sample_ai4: dict[str, Any] = ai4privacy_dataset["train"][0]
print(f"\nSample ai4privacy record keys: {list(sample_ai4.keys())}")

ai4privacy English-only dataset loaded:
  Train split: 120,533 examples
  Validation split: 30,160 examples

Original features: ['source_text', 'masked_text', 'privacy_mask', 'split', 'uid', 'language', 'region', 'script', 'mbert_tokens', 'mbert_token_classes']

Sample ai4privacy record keys: ['source_text', 'masked_text', 'privacy_mask', 'split', 'uid', 'language', 'region', 'script', 'mbert_tokens', 'mbert_token_classes']


## Load Validated Synthetic Samples

Load the GPT-5.1 validated synthetic samples from JSONL format.


In [28]:
synthetic_samples: list[dict[str, Any]] = []

with open(SYNTHETIC_VALIDATED_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            synthetic_samples.append(json.loads(line))

print(f"Synthetic validated samples loaded: {len(synthetic_samples):,} examples")

# Display distribution by feature dimension
dimension_counts: Counter[str] = Counter(
    s.get("feature_dimension", "unknown") for s in synthetic_samples
)
print("\nDistribution by feature dimension:")
for dim, count in sorted(dimension_counts.items()):
    pct = count / len(synthetic_samples) * 100
    print(f"  {dim}: {count:,} ({pct:.1f}%)")

# Display sample record structure
print(f"\nSample synthetic record keys: {list(synthetic_samples[0].keys())}")

Synthetic validated samples loaded: 6,002 examples

Distribution by feature dimension:
  adversarial: 580 (9.7%)
  basic: 1,237 (20.6%)
  contextual: 1,001 (16.7%)
  evolving: 943 (15.7%)
  multilingual: 1,146 (19.1%)
  noisy: 1,095 (18.2%)

Sample synthetic record keys: ['text', 'entities', 'feature_dimension', 'seed_pii_type', 'seed_pii_value', 'seed_pii_locale', 'scenario', 'type_variant', 'generation_id', 'timestamp']


## Stratified 80-20 Split of Synthetic Data

Perform stratified 80-20 train/test split on synthetic data. Stratification ensures balanced representation of each
feature dimension in both splits.


In [29]:
# Extract feature dimensions for stratification
stratify_labels: list[str] = [
    s.get("feature_dimension", "unknown") for s in synthetic_samples
]

# Create indices for splitting
indices: list[int] = list(range(len(synthetic_samples)))

# Perform stratified split
train_indices, test_indices = train_test_split(
    indices,
    train_size=SYNTHETIC_TRAIN_RATIO,
    test_size=SYNTHETIC_TEST_RATIO,
    random_state=RANDOM_SEED,
    stratify=stratify_labels,
)

# Split the samples
synthetic_train: list[dict[str, Any]] = [synthetic_samples[i] for i in train_indices]
synthetic_test: list[dict[str, Any]] = [synthetic_samples[i] for i in test_indices]

print(f"Synthetic data split complete:")
print(f"  Train: {len(synthetic_train):,} samples ({len(synthetic_train)/len(synthetic_samples)*100:.1f}%)")
print(f"  Test: {len(synthetic_test):,} samples ({len(synthetic_test)/len(synthetic_samples)*100:.1f}%)")

# Verify stratification preserved dimension distribution
print("\nVerify stratification (train split):")
train_dims: Counter[str] = Counter(s.get("feature_dimension") for s in synthetic_train)
for dim, count in sorted(train_dims.items()):
    pct = count / len(synthetic_train) * 100
    print(f"  {dim}: {count:,} ({pct:.1f}%)")

print("\nVerify stratification (test split):")
test_dims: Counter[str] = Counter(s.get("feature_dimension") for s in synthetic_test)
for dim, count in sorted(test_dims.items()):
    pct = count / len(synthetic_test) * 100
    print(f"  {dim}: {count:,} ({pct:.1f}%)")

Synthetic data split complete:
  Train: 4,801 samples (80.0%)
  Test: 1,201 samples (20.0%)

Verify stratification (train split):
  adversarial: 464 (9.7%)
  basic: 989 (20.6%)
  contextual: 801 (16.7%)
  evolving: 754 (15.7%)
  multilingual: 917 (19.1%)
  noisy: 876 (18.2%)

Verify stratification (test split):
  adversarial: 116 (9.7%)
  basic: 248 (20.6%)
  contextual: 200 (16.7%)
  evolving: 189 (15.7%)
  multilingual: 229 (19.1%)
  noisy: 219 (18.2%)


## Schema Conversion Functions

Convert between synthetic and ai4privacy dataset schemas.

The ai4privacy schema uses: - source_text: Original text with PII - privacy_mask: List of {label, start, end, value,
label_index} - mbert_tokens/mbert_token_classes: Pre-tokenized for mBERT

The synthetic schema uses: - text: Original text with PII - entities: List of {start, end, label, text}

We'll create a unified schema that preserves all information from both sources while maintaining compatibility for
downstream tokenization with DeBERTaV3.


In [30]:
def convert_synthetic_to_unified(sample: dict[str, Any]) -> dict[str, Any]:
    """
    Convert a synthetic sample to the unified schema format.
    
    Args:
        sample: A synthetic sample dictionary with keys like 'text', 'entities',
                'feature_dimension', etc.
    
    Returns:
        A dictionary in unified schema format compatible with ai4privacy records.
    """
    # Convert entity spans from synthetic format to ai4privacy-compatible format
    privacy_mask: list[dict[str, Any]] = []
    
    entities = sample.get("entities", [])
    label_counters: Counter[str] = Counter()
    
    for entity in entities:
        label: str = entity.get("label", "UNKNOWN")
        label_counters[label] += 1
        
        privacy_mask.append({
            "label": label,
            "start": int(entity.get("start", 0)),
            "end": int(entity.get("end", 0)),
            "value": str(entity.get("text", "")),
            "label_index": int(label_counters[label]),
        })
    
    return {
        # Core fields
        "source_text": str(sample.get("text", "")),
        "privacy_mask": privacy_mask,
        
        # Synthetic-specific metadata (ensure string types, handle None)
        "feature_dimension": str(sample.get("feature_dimension", "")),
        "seed_pii_type": str(sample.get("seed_pii_type", "")),
        "seed_pii_value": str(sample.get("seed_pii_value", "")),
        "seed_pii_locale": str(sample.get("seed_pii_locale") or ""),
        "scenario": str(sample.get("scenario", "")),
        "type_variant": str(sample.get("type_variant", "")),
        "generation_id": str(sample.get("generation_id", "")),
        
        # Source identification
        "data_source": "synthetic",
        
        # Language metadata
        "language": "en",
        "region": "",
        "script": "Latn",
    }


def convert_ai4privacy_to_unified(sample: dict[str, Any]) -> dict[str, Any]:
    """
    Convert an ai4privacy sample to the unified schema format.
    
    Args:
        sample: An ai4privacy sample dictionary with keys like 'source_text',
                'privacy_mask', 'mbert_tokens', etc.
    
    Returns:
        A dictionary in unified schema format.
    """
    # Ensure privacy_mask entries have correct types
    privacy_mask: list[dict[str, Any]] = []
    for entry in sample.get("privacy_mask", []):
        privacy_mask.append({
            "label": str(entry.get("label", "")),
            "start": int(entry.get("start", 0)),
            "end": int(entry.get("end", 0)),
            "value": str(entry.get("value", "")),
            "label_index": int(entry.get("label_index", 0)),
        })
    
    return {
        # Core fields
        "source_text": str(sample.get("source_text", "")),
        "privacy_mask": privacy_mask,
        
        # Synthetic-specific metadata (empty for ai4privacy)
        "feature_dimension": "",
        "seed_pii_type": "",
        "seed_pii_value": "",
        "seed_pii_locale": "",
        "scenario": "",
        "type_variant": "",
        "generation_id": "",
        
        # Source identification
        "data_source": "ai4privacy",
        
        # Language metadata
        "language": str(sample.get("language", "en")),
        "region": str(sample.get("region") or ""),
        "script": str(sample.get("script") or ""),
    }


# Test the conversion functions
print("Testing schema conversion functions...")

test_synthetic = synthetic_samples[0]
converted_synthetic = convert_synthetic_to_unified(test_synthetic)
print(f"\nSynthetic → Unified:")
print(f"  source_text (preview): {converted_synthetic['source_text'][:80]}...")
print(f"  privacy_mask count: {len(converted_synthetic['privacy_mask'])}")
print(f"  data_source: {converted_synthetic['data_source']}")

test_ai4 = ai4privacy_dataset["train"][0]
converted_ai4 = convert_ai4privacy_to_unified(test_ai4)
print(f"\nai4privacy → Unified:")
print(f"  source_text (preview): {converted_ai4['source_text'][:80]}...")
print(f"  privacy_mask count: {len(converted_ai4['privacy_mask'])}")
print(f"  data_source: {converted_ai4['data_source']}")

Testing schema conversion functions...

Synthetic → Unified:
  source_text (preview): Dear Mr. Raj Patel,

Thank you for your visa application to Canada. Here are you...
  privacy_mask count: 5
  data_source: synthetic

ai4privacy → Unified:
  source_text (preview): To-do list for 4th August 1942: meet with Brandy Haroon at 10:17 to discuss the ...
  privacy_mask count: 4
  data_source: ai4privacy


## Convert All Samples to Unified Format

Convert all samples from both sources to the unified schema format.


In [31]:
from tqdm.auto import tqdm

# Convert synthetic samples
print("Converting synthetic train samples to unified format...")
synthetic_train_unified: list[dict[str, Any]] = [
    convert_synthetic_to_unified(s) for s in tqdm(synthetic_train, desc="Synthetic train")
]

print("Converting synthetic test samples to unified format...")
synthetic_test_unified: list[dict[str, Any]] = [
    convert_synthetic_to_unified(s) for s in tqdm(synthetic_test, desc="Synthetic test")
]

# Convert ai4privacy samples
print("\nConverting ai4privacy samples to unified format...")

ai4_train_unified: list[dict[str, Any]] = [
    convert_ai4privacy_to_unified(sample) 
    for sample in tqdm(ai4privacy_dataset["train"], desc="ai4privacy train")
]

ai4_val_unified: list[dict[str, Any]] = [
    convert_ai4privacy_to_unified(sample) 
    for sample in tqdm(ai4privacy_dataset["validation"], desc="ai4privacy validation")
]

print("\nConversion complete:")
print(f"  Synthetic train (unified): {len(synthetic_train_unified):,}")
print(f"  Synthetic test (unified): {len(synthetic_test_unified):,}")
print(f"  ai4privacy train (unified): {len(ai4_train_unified):,}")
print(f"  ai4privacy validation (unified): {len(ai4_val_unified):,}")

Converting synthetic train samples to unified format...


Synthetic train:   0%|          | 0/4801 [00:00<?, ?it/s]

Converting synthetic test samples to unified format...


Synthetic test:   0%|          | 0/1201 [00:00<?, ?it/s]


Converting ai4privacy samples to unified format...


ai4privacy train:   0%|          | 0/120533 [00:00<?, ?it/s]

ai4privacy validation:   0%|          | 0/30160 [00:00<?, ?it/s]


Conversion complete:
  Synthetic train (unified): 4,801
  Synthetic test (unified): 1,201
  ai4privacy train (unified): 120,533
  ai4privacy validation (unified): 30,160


## Combine and Create Dataset with Explicit Features

Combine datasets and create HuggingFace Dataset:

-   Combined train = ai4privacy train + synthetic train
-   Combined test = ai4privacy validation + synthetic test

Note: We let the datasets library infer the schema automatically for the nested privacy_mask structure, then verify the
inferred types match expectations. This avoids encoding issues with complex nested Sequence features.


In [32]:
# Combine train sets
combined_train_records: list[dict[str, Any]] = ai4_train_unified + synthetic_train_unified

# Combine test sets
combined_test_records: list[dict[str, Any]] = ai4_val_unified + synthetic_test_unified

print("Combined dataset statistics:")
print(f"\n  TRAIN SET:")
print(f"    ai4privacy: {len(ai4_train_unified):,}")
print(f"    synthetic: {len(synthetic_train_unified):,}")
print(f"    combined: {len(combined_train_records):,}")

print(f"\n  TEST SET:")
print(f"    ai4privacy: {len(ai4_val_unified):,}")
print(f"    synthetic: {len(synthetic_test_unified):,}")
print(f"    combined: {len(combined_test_records):,}")

# Create Dataset objects WITHOUT explicit features (let datasets infer schema)
# This handles nested structures like privacy_mask more reliably
print("\nCreating HuggingFace Datasets (schema will be inferred)...")

combined_train_dataset: Dataset = Dataset.from_list(combined_train_records)
combined_test_dataset: Dataset = Dataset.from_list(combined_test_records)

# Create DatasetDict
combined_dataset: DatasetDict = DatasetDict({
    "train": combined_train_dataset,
    "test": combined_test_dataset,
})

print(f"\n{combined_dataset}")

# Verify inferred schema matches expectations
print(f"\nInferred Features (verify these match expectations):")
for feature_name, feature_type in combined_dataset["train"].features.items():
    print(f"  {feature_name}: {feature_type}")

# Validate critical fields have correct types
train_features = combined_dataset["train"].features
assert "source_text" in train_features, "Missing source_text field"
assert "privacy_mask" in train_features, "Missing privacy_mask field"
assert "data_source" in train_features, "Missing data_source field"
print("\n✓ Schema validation passed")

Combined dataset statistics:

  TRAIN SET:
    ai4privacy: 120,533
    synthetic: 4,801
    combined: 125,334

  TEST SET:
    ai4privacy: 30,160
    synthetic: 1,201
    combined: 31,361

Creating HuggingFace Datasets (schema will be inferred)...

DatasetDict({
    train: Dataset({
        features: ['source_text', 'privacy_mask', 'feature_dimension', 'seed_pii_type', 'seed_pii_value', 'seed_pii_locale', 'scenario', 'type_variant', 'generation_id', 'data_source', 'language', 'region', 'script'],
        num_rows: 125334
    })
    test: Dataset({
        features: ['source_text', 'privacy_mask', 'feature_dimension', 'seed_pii_type', 'seed_pii_value', 'seed_pii_locale', 'scenario', 'type_variant', 'generation_id', 'data_source', 'language', 'region', 'script'],
        num_rows: 31361
    })
})

Inferred Features (verify these match expectations):
  source_text: Value('string')
  privacy_mask: List({'end': Value('int64'), 'label': Value('string'), 'label_index': Value('int64'), 'start'

## Pre-Upload Validation


In [33]:
print("=" * 60)
print("PRE-UPLOAD VALIDATION")
print("=" * 60)

# Test first example from each split
print("\n1. Testing first example loads correctly...")
try:
    train_example = combined_dataset["train"][0]
    print(f"   ✓ Train example loaded successfully")
    print(f"     - source_text length: {len(train_example['source_text'])} chars")
    print(f"     - privacy_mask entries: {len(train_example['privacy_mask'])}")
    print(f"     - data_source: {train_example['data_source']}")
    
    test_example = combined_dataset["test"][0]
    print(f"   ✓ Test example loaded successfully")
except Exception as e:
    print(f"   ✗ Error loading example: {e}")

# Check dataset sizes
print("\n2. Checking dataset sizes...")
print(f"   Train: {len(combined_dataset['train']):,} examples")
print(f"   Test: {len(combined_dataset['test']):,} examples")

# Verify features are preserved
print("\n3. Verifying features schema...")
expected_features = set(UNIFIED_FEATURES.keys())
actual_features = set(combined_dataset["train"].features.keys())
if expected_features == actual_features:
    print(f"   ✓ All {len(expected_features)} features present")
else:
    missing = expected_features - actual_features
    extra = actual_features - expected_features
    if missing:
        print(f"   ✗ Missing features: {missing}")
    if extra:
        print(f"   ⚠ Extra features: {extra}")

# Validate data source distribution
print("\n4. Validating data source distribution...")
train_sources: Counter[str] = Counter(
    combined_dataset["train"]["data_source"]
)
test_sources: Counter[str] = Counter(
    combined_dataset["test"]["data_source"]
)
print(f"   Train: {dict(train_sources)}")
print(f"   Test: {dict(test_sources)}")

# Check for empty/null values in critical fields
print("\n5. Checking for empty critical fields...")
empty_source_text_train = sum(
    1 for x in combined_dataset["train"]["source_text"] if not x.strip()
)
empty_source_text_test = sum(
    1 for x in combined_dataset["test"]["source_text"] if not x.strip()
)
if empty_source_text_train == 0 and empty_source_text_test == 0:
    print(f"   ✓ No empty source_text fields")
else:
    print(f"   ⚠ Empty source_text: {empty_source_text_train} train, {empty_source_text_test} test")

print("\n" + "=" * 60)
print("Validation complete. Review any warnings above before upload.")
print("=" * 60)

PRE-UPLOAD VALIDATION

1. Testing first example loads correctly...
   ✓ Train example loaded successfully
     - source_text length: 137 chars
     - privacy_mask entries: 4
     - data_source: ai4privacy
   ✓ Test example loaded successfully

2. Checking dataset sizes...
   Train: 125,334 examples
   Test: 31,361 examples

3. Verifying features schema...
   ✓ All 13 features present

4. Validating data source distribution...
   Train: {'ai4privacy': 120533, 'synthetic': 4801}
   Test: {'ai4privacy': 30160, 'synthetic': 1201}

5. Checking for empty critical fields...
   ⚠ Empty source_text: 7 train, 0 test

Validation complete. Review any warnings above before upload.


## Investigate and Clean Empty Records

Empty text samples provide no training signal and can cause issues with tokenization and loss computation during
training.


In [34]:
# Find indices of empty source_text in train set
empty_train_indices: list[int] = [
    i for i, text in enumerate(combined_dataset["train"]["source_text"])
    if not text.strip()
]

print(f"Found {len(empty_train_indices)} empty source_text records in train set")

# Investigate the empty records
if empty_train_indices:
    print("\nInvestigating empty records:")
    print("-" * 60)
    
    for idx in empty_train_indices:
        record = combined_dataset["train"][idx]
        print(f"\n  Index {idx}:")
        print(f"    data_source: {record['data_source']}")
        print(f"    source_text: '{record['source_text']}' (length: {len(record['source_text'])})")
        print(f"    privacy_mask entries: {len(record['privacy_mask'])}")
        
        # Show additional context for synthetic records
        if record["data_source"] == "synthetic":
            print(f"    feature_dimension: {record['feature_dimension']}")
            print(f"    generation_id: {record['generation_id']}")
        else:
            print(f"    language: {record['language']}")
            print(f"    region: {record['region']}")

# Count by source
empty_by_source: Counter[str] = Counter(
    combined_dataset["train"][i]["data_source"] for i in empty_train_indices
)
print(f"\nEmpty records by source: {dict(empty_by_source)}")

# Remove empty records from train set
print("\n" + "-" * 60)
print("Removing empty records...")

# Create mask for non-empty records
non_empty_mask: list[bool] = [
    bool(text.strip()) for text in combined_dataset["train"]["source_text"]
]

# Filter train set
train_before = len(combined_dataset["train"])
combined_dataset["train"] = combined_dataset["train"].filter(
    lambda example: bool(example["source_text"].strip()),
    desc="Filtering empty source_text",
)
train_after = len(combined_dataset["train"])

print(f"\nTrain set: {train_before:,} → {train_after:,} ({train_before - train_after} removed)")

# Verify no empty records remain
remaining_empty = sum(
    1 for text in combined_dataset["train"]["source_text"] if not text.strip()
)
if remaining_empty == 0:
    print("✓ All empty records removed from train set")
else:
    print(f"⚠ Still have {remaining_empty} empty records (unexpected)")

# Also check test set (should be 0 based on earlier validation)
test_empty = sum(
    1 for text in combined_dataset["test"]["source_text"] if not text.strip()
)
print(f"✓ Test set has {test_empty} empty records (expected: 0)")

Found 7 empty source_text records in train set

Investigating empty records:
------------------------------------------------------------

  Index 1894:
    data_source: ai4privacy
    source_text: '' (length: 0)
    privacy_mask entries: 0
    language: en
    region: GB

  Index 14629:
    data_source: ai4privacy
    source_text: '' (length: 0)
    privacy_mask entries: 0
    language: en
    region: US

  Index 20189:
    data_source: ai4privacy
    source_text: '' (length: 0)
    privacy_mask entries: 0
    language: en
    region: CA

  Index 45145:
    data_source: ai4privacy
    source_text: '' (length: 0)
    privacy_mask entries: 0
    language: en
    region: IN

  Index 62063:
    data_source: ai4privacy
    source_text: '' (length: 0)
    privacy_mask entries: 0
    language: en
    region: US

  Index 108065:
    data_source: ai4privacy
    source_text: '' (length: 0)
    privacy_mask entries: 0
    language: en
    region: US

  Index 108066:
    data_source: ai4privacy
 

Filtering empty source_text:   0%|          | 0/125334 [00:00<?, ? examples/s]


Train set: 125,334 → 125,327 (7 removed)
✓ All empty records removed from train set
✓ Test set has 0 empty records (expected: 0)


## Save Locally as Parquet


In [35]:
# Create output directory
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Save as Parquet files (recommended by HF guide)
parquet_train_path = OUTPUT_PATH / "train.parquet"
parquet_test_path = OUTPUT_PATH / "test.parquet"

print("Saving dataset as Parquet (recommended format)...")

combined_dataset["train"].to_parquet(str(parquet_train_path))
print(f"  ✓ Train: {parquet_train_path} ({parquet_train_path.stat().st_size / 1024 / 1024:.1f} MB)")

combined_dataset["test"].to_parquet(str(parquet_test_path))
print(f"  ✓ Test: {parquet_test_path} ({parquet_test_path.stat().st_size / 1024 / 1024:.1f} MB)")

# Also save DatasetDict for convenience (Arrow format)
combined_dataset.save_to_disk(str(OUTPUT_PATH / "arrow"))
print(f"  ✓ Arrow format: {OUTPUT_PATH / 'arrow'}")

print("\nTo reload locally:")
print(f"  # From Parquet:")
print(f"  from datasets import load_dataset")
print(f"  dataset = load_dataset('parquet', data_files={{'train': '{parquet_train_path}', 'test': '{parquet_test_path}'}})")
print(f"\n  # From Arrow:")
print(f"  from datasets import load_from_disk")
print(f"  dataset = load_from_disk('{OUTPUT_PATH / 'arrow'}')")

Saving dataset as Parquet (recommended format)...


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  ✓ Train: data\combined_dataset\train.parquet (14.9 MB)


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  ✓ Test: data\combined_dataset\test.parquet (3.7 MB)


Saving the dataset (0/1 shards):   0%|          | 0/125327 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/31361 [00:00<?, ? examples/s]

  ✓ Arrow format: data\combined_dataset\arrow

To reload locally:
  # From Parquet:
  from datasets import load_dataset
  dataset = load_dataset('parquet', data_files={'train': 'data\combined_dataset\train.parquet', 'test': 'data\combined_dataset\test.parquet'})

  # From Arrow:
  from datasets import load_from_disk
  dataset = load_from_disk('data\combined_dataset\arrow')


## Generate Dataset Card

Generate Dataset Card (README.md) for the combined dataset.


In [36]:
from datetime import datetime

DATASET_CARD_TEMPLATE = '''---
license: mit
task_categories:
  - token-classification
language:
  - en
tags:
  - pii
  - ner
  - privacy
  - synthetic-data
size_categories:
  - 100K<n<1M
configs:
  - config_name: default
    data_files:
      - split: train
        path: "train.parquet"
      - split: test
        path: "test.parquet"
---

# PII Detection Combined Dataset

Combined dataset for PII (Personally Identifiable Information) detection,
merging the ai4privacy English-only subset with synthetically generated
challenging examples targeting NER failure modes.

## Dataset Description

This dataset combines two sources:

1. **ai4privacy/open-pii-masking-500k** (English subset): {ai4_train:,} train / {ai4_test:,} test examples
2. **Synthetic data** (GPT-5.1 validated): {syn_train:,} train / {syn_test:,} test examples

**Total**: {total_train:,} train / {total_test:,} test examples

### Synthetic Data Feature Dimensions

The synthetic data specifically targets six NER failure mode dimensions
from Singh & Narayanan (2025) "Unmasking the Reality of PII Masking Models":

| Dimension | Train Count | Description |
|-----------|-------------|-------------|
{dimension_table}

## Dataset Schema

| Field | Type | Description |
|-------|------|-------------|
| `source_text` | string | Original text containing PII entities |
| `privacy_mask` | list | Entity annotations with label, start, end, value |
| `data_source` | string | Either "ai4privacy" or "synthetic" |
| `feature_dimension` | string | NER challenge dimension (synthetic only) |
| `language` | string | Language code (always "en") |

## Usage
```python
from datasets import load_dataset

# Load from HuggingFace Hub
dataset = load_dataset("{repo_id}")

# Or load from local Parquet files
dataset = load_dataset("parquet", data_files={{
    "train": "train.parquet",
    "test": "test.parquet"
}})

# Access examples
print(dataset["train"][0])
```

## Citation

If you use this dataset, please cite:
```bibtex
@misc{{pii_combined_dataset_{year},
  title={{PII Detection Combined Dataset}},
  year={{{year}}},
  publisher={{Hugging Face}},
  note={{Combines ai4privacy English subset with synthetic challenging examples}}
}}
```

## License

MIT License

## Dataset Creation

- **Created**: {date}
- **ai4privacy source**: ai4privacy/open-pii-masking-500k-ai4privacy
- **Synthetic validation**: OpenAI GPT-5.1
- **Split strategy**: 80/20 stratified split on synthetic data
'''

# Generate dimension table
dimension_table_rows: list[str] = []
for dim, count in sorted(train_dims.items()):
    descriptions = {
        "basic": "Standard, well-formatted entities",
        "contextual": "Ambiguous entities requiring context",
        "noisy": "Real-world text imperfections",
        "evolving": "Modern/emerging PII formats",
        "multilingual": "International formats in English",
        "adversarial": "Intentionally deceptive patterns",
    }
    desc = descriptions.get(dim, "Unknown dimension")
    dimension_table_rows.append(f"| {dim} | {count:,} | {desc} |")

dimension_table = "\n".join(dimension_table_rows)

# Fill in template
readme_content = DATASET_CARD_TEMPLATE.format(
    ai4_train=len(ai4_train_unified),
    ai4_test=len(ai4_val_unified),
    syn_train=len(synthetic_train_unified),
    syn_test=len(synthetic_test_unified),
    total_train=len(combined_train_records),
    total_test=len(combined_test_records),
    dimension_table=dimension_table,
    repo_id=HF_REPO_ID or "your-username/pii-detection-combined",
    year=datetime.now().year,
    date=datetime.now().strftime("%Y-%m-%d"),
)

# Save README.md
readme_path = OUTPUT_PATH / "README.md"
readme_path.write_text(readme_content, encoding="utf-8")

print(f"Dataset card saved: {readme_path}")
print("\nPreview:")
print("-" * 60)
print(readme_content[:1500] + "...")

Dataset card saved: data\combined_dataset\README.md

Preview:
------------------------------------------------------------
---
license: mit
task_categories:
  - token-classification
language:
  - en
tags:
  - pii
  - ner
  - privacy
  - synthetic-data
size_categories:
  - 100K<n<1M
configs:
  - config_name: default
    data_files:
      - split: train
        path: "train.parquet"
      - split: test
        path: "test.parquet"
---

# PII Detection Combined Dataset

Combined dataset for PII (Personally Identifiable Information) detection,
merging the ai4privacy English-only subset with synthetically generated
challenging examples targeting NER failure modes.

## Dataset Description

This dataset combines two sources:

1. **ai4privacy/open-pii-masking-500k** (English subset): 120,533 train / 30,160 test examples
2. **Synthetic data** (GPT-5.1 validated): 4,801 train / 1,201 test examples

**Total**: 125,334 train / 31,361 test examples

### Synthetic Data Feature Dimensions

The synthe

## Push to HuggingFace Hub


In [37]:
if HF_REPO_ID:
    print(f"Pushing dataset to HuggingFace Hub: {HF_REPO_ID}")
    print(f"  Private: {HF_PRIVATE}")
    print(f"  Max shard size: {MAX_SHARD_SIZE}")
    
    # Push with memory-efficient settings
    combined_dataset.push_to_hub(
        HF_REPO_ID,
        private=HF_PRIVATE,
        max_shard_size=MAX_SHARD_SIZE,
    )
    
    print(f"\n✓ Dataset uploaded successfully!")
    print(f"  View at: https://huggingface.co/datasets/{HF_REPO_ID}")
    print(f"\nTo load from Hub:")
    print(f"  from datasets import load_dataset")
    print(f"  dataset = load_dataset('{HF_REPO_ID}')")
else:
    print("HF_REPO_ID not configured. Skipping Hub upload.")
    print("\nTo upload later, set HF_REPO_ID and run:")
    print("  combined_dataset.push_to_hub('your-username/dataset-name')")
    print("\nOr upload the Parquet files directly via the Hub UI:")
    print(f"  {OUTPUT_PATH / 'train.parquet'}")
    print(f"  {OUTPUT_PATH / 'test.parquet'}")
    print(f"  {OUTPUT_PATH / 'README.md'}")

Pushing dataset to HuggingFace Hub: Ari-S-123/better-english-pii-anonymizer
  Private: False
  Max shard size: 500MB


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


✓ Dataset uploaded successfully!
  View at: https://huggingface.co/datasets/Ari-S-123/better-english-pii-anonymizer

To load from Hub:
  from datasets import load_dataset
  dataset = load_dataset('Ari-S-123/better-english-pii-anonymizer')


## Final Summary


In [38]:
print("=" * 80)
print("DATASET COMBINATION SUMMARY")
print("=" * 80)

print("\n📊 INPUT DATA:")
print(f"   ai4privacy train: {len(ai4privacy_dataset['train']):,}")
print(f"   ai4privacy validation: {len(ai4privacy_dataset['validation']):,}")
print(f"   synthetic validated: {len(synthetic_samples):,}")

print("\n🔀 SYNTHETIC SPLIT (stratified by dimension):")
print(f"   Train ({SYNTHETIC_TRAIN_RATIO:.0%}): {len(synthetic_train):,}")
print(f"   Test ({SYNTHETIC_TEST_RATIO:.0%}): {len(synthetic_test):,}")

print("\n✅ COMBINED OUTPUT:")
print(f"   Train total: {len(combined_train_records):,}")
print(f"     ├─ ai4privacy: {len(ai4_train_unified):,} ({len(ai4_train_unified)/len(combined_train_records)*100:.1f}%)")
print(f"     └─ synthetic: {len(synthetic_train_unified):,} ({len(synthetic_train_unified)/len(combined_train_records)*100:.1f}%)")
print(f"   Test total: {len(combined_test_records):,}")
print(f"     ├─ ai4privacy: {len(ai4_val_unified):,} ({len(ai4_val_unified)/len(combined_test_records)*100:.1f}%)")
print(f"     └─ synthetic: {len(synthetic_test_unified):,} ({len(synthetic_test_unified)/len(combined_test_records)*100:.1f}%)")

print("\n📁 OUTPUT FILES:")
print(f"   Directory: {OUTPUT_PATH}")
print(f"   ├─ train.parquet (recommended format)")
print(f"   ├─ test.parquet")
print(f"   ├─ README.md (dataset card)")
print(f"   └─ arrow/ (DatasetDict format)")

print("\n📈 SYNTHETIC DIMENSION DISTRIBUTION IN TRAIN:")
for dim, count in sorted(train_dims.items()):
    print(f"   {dim}: {count:,}")

if HF_REPO_ID:
    print(f"\n🌐 HUGGINGFACE HUB:")
    print(f"   Repository: https://huggingface.co/datasets/{HF_REPO_ID}")

print("\n" + "=" * 80)
print("✓ Dataset combination complete. Ready for tokenization and training.")
print("=" * 80)

DATASET COMBINATION SUMMARY

📊 INPUT DATA:
   ai4privacy train: 120,533
   ai4privacy validation: 30,160
   synthetic validated: 6,002

🔀 SYNTHETIC SPLIT (stratified by dimension):
   Train (80%): 4,801
   Test (20%): 1,201

✅ COMBINED OUTPUT:
   Train total: 125,334
     ├─ ai4privacy: 120,533 (96.2%)
     └─ synthetic: 4,801 (3.8%)
   Test total: 31,361
     ├─ ai4privacy: 30,160 (96.2%)
     └─ synthetic: 1,201 (3.8%)

📁 OUTPUT FILES:
   Directory: data\combined_dataset
   ├─ train.parquet (recommended format)
   ├─ test.parquet
   ├─ README.md (dataset card)
   └─ arrow/ (DatasetDict format)

📈 SYNTHETIC DIMENSION DISTRIBUTION IN TRAIN:
   adversarial: 464
   basic: 989
   contextual: 801
   evolving: 754
   multilingual: 917
   noisy: 876

🌐 HUGGINGFACE HUB:
   Repository: https://huggingface.co/datasets/Ari-S-123/better-english-pii-anonymizer

✓ Dataset combination complete. Ready for tokenization and training.
